## Kaggle competition -- Xingjia Wu

In [1]:
#from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import pandas as pd
import os
# import cv2
from sklearn.preprocessing import LabelEncoder
from scipy import misc
from sklearn.cross_validation import train_test_split

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import itemfreq

Using Theano backend.


In [2]:
np.random.seed(1337)  # for reproducibility

### Read in training dataset from 'images'

In [3]:
file_loc = r'/root/sharedfolder/images'
file_name = r'/root/sharedfolder/new_legend.csv'

In [4]:
e = pd.read_csv(file_name)
edata = e[['image', 'emotion', 'new_emotion']]

In [5]:
edata['new_emotion'].value_counts()

neutral      5991
happiness    5041
angry         418
fear          343
sadness       256
dtype: int64

In [6]:
# input image dimensions
pixel = 32 # image resize to 32X32
imageResize = (pixel, pixel)

In [7]:
n = len(edata)-925 # exclude images with emotion but no pictures and the first four colorful one
X = np.empty(shape = [n-4, pixel, pixel]) # exclude first four
Y = []

In [8]:
m = 0
for i in range(4,n):
#     try:
        f = os.path.join(file_loc,edata['image'][i])
        frame = misc.imread(f)
        frameResize = misc.imresize(frame, imageResize) # Resize into 96
        X[m] = frameResize
        Y.append(edata['new_emotion'][i])
        m += 1
#     except:
#         m += 1
#         print(edata['image'][i])

### Split dataset into train (75%) and test (25%) sets


In [9]:
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.25, random_state=42)

In [10]:
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded = encoder.transform(Y_train)
Y_train = np_utils.to_categorical(encoded)

encoder = LabelEncoder()
encoder.fit(Y_test)
encoded = encoder.transform(Y_test)
Y_test = np_utils.to_categorical(encoded)

In [11]:
encoder.inverse_transform([0, 1, 2, 3, 4])

array(['angry', 'fear', 'happiness', 'neutral', 'sadness'], 
      dtype='|S9')

In [12]:
img_rows, img_cols = pixel, pixel
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

In [13]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print "*********************************"
print('Y_train shape:', Y_train.shape)
print(Y_train.shape[0], 'train samples')
print(Y_test.shape[0], 'test samples')

('X_train shape:', (8340, 1, 32, 32))
(8340, 'train samples')
(2780, 'test samples')
*********************************
('Y_train shape:', (8340, 5))
(8340, 'train samples')
(2780, 'test samples')


### CNN model

In [14]:
input_shape = (1, img_rows, img_cols)
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
batch_size = 128
nb_classes = 5 # was 6 for first 2000 images
nb_epoch = 12

In [15]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode = 'valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    return model

In [16]:
model = create_model()

### Training model with inline augmentation

In [17]:
from keras.preprocessing.image import ImageDataGenerator
import time
train_datagen = ImageDataGenerator(
    featurewise_std_normalization=False,
    rotation_range = 20,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True)

train_generator = train_datagen.flow(X_train, Y_train, batch_size = 256)

# model.reset_states()
start_time = time.time()
history = model.fit_generator(
    train_generator,
    samples_per_epoch=len(X_train)*20,
    nb_epoch=12,
    validation_data=(X_test, Y_test))

print "Training duration : {0}".format(time.time() - start_time)
score = model.evaluate(X_test, Y_test, batch_size = batch_size)

print "Network's test score [loss, accuracy]: {0}".format(score)
print "Network's time {0} minutes".format((time.time() - start_time)/60)

Epoch 1/12
166800/166800 [==============================] - 1203s - loss: 0.8094 - acc: 0.6570 - val_loss: 0.5404 - val_acc: 0.8144
Epoch 2/12
166800/166800 [==============================] - 1211s - loss: 0.6234 - acc: 0.7740 - val_loss: 0.4603 - val_acc: 0.8385
Epoch 3/12
166800/166800 [==============================] - 1213s - loss: 0.5572 - acc: 0.8033 - val_loss: 0.4243 - val_acc: 0.8583
Epoch 4/12
166800/166800 [==============================] - 1215s - loss: 0.5202 - acc: 0.8175 - val_loss: 0.4042 - val_acc: 0.8676
Epoch 5/12
166800/166800 [==============================] - 1246s - loss: 0.4957 - acc: 0.8277 - val_loss: 0.3911 - val_acc: 0.8691
Epoch 6/12
166800/166800 [==============================] - 1271s - loss: 0.4792 - acc: 0.8334 - val_loss: 0.3765 - val_acc: 0.8773
Epoch 7/12
166800/166800 [==============================] - 1273s - loss: 0.4588 - acc: 0.8404 - val_loss: 0.3803 - val_acc: 0.8788
Epoch 8/12
166800/166800 [==============================] - 1322s - loss: 0.

### Save model

In [18]:
from keras.models import model_from_json

model_json = model.to_json()

with open("model_unbalanced_aug.json", "w") as json_file:

    json_file.write(model_json)

# serialize weights to HDF5

model.save_weights("model_unbalanced_aug.h5")

print("Saved model to disk") 

Saved model to disk


### Read in test image preprocessed by OpenCV

In [19]:
from scipy import misc

In [20]:
# Modified data
indir = r'/root/sharedfolder/testFace'
piclist = os.listdir(indir)

In [21]:
pixel = 32
n = len(piclist)
X = np.empty(shape = [n, pixel, pixel])

In [22]:
for i in range(n):
        f = os.path.join(indir, piclist[i])
        frame = misc.imread(f,  flatten=True)
        frame = misc.imresize(frame,(pixel, pixel) )
        X[i]= frame

In [23]:
X_test = X.reshape(X.shape[0], 1, pixel, pixel)
X_test = X_test.astype('float32')
X_test /= 255
print(X_test.shape[0], 'test samples')

(263, 'test samples')


### Load model

In [24]:
json_file = open('model_unbalanced_aug.json', 'r')

loaded_model_json = json_file.read()

json_file.close()

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("model_unbalanced_aug.h5")

loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

### Prediction

In [25]:
predicted = loaded_model.predict(X_test, verbose = 0)
predEmotion = np.argmax(predicted, axis=1)

In [26]:
label = pd.concat([pd.Series(piclist), pd.Series(predEmotion)], axis=1)
label.columns = ['Image', 'code']

In [27]:
ecode = {0:'anger', 1:'fear', 2:'happiness', 3:'neutral', 4:'sadness'}
edf = pd.DataFrame.from_dict(ecode, orient='index').reset_index()
edf = edf.rename(columns={'index': 'code', 0: 'Emotion'})

In [28]:
resultTable = pd.merge(label, edf, on = ['code'], how='inner')
result = resultTable.drop(['code'], 1)

# Write out the csv
result.to_csv('evaluation.csv', index=False)